In [1]:

import pandas as pd

from datetime import date
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

import datetime
from datetime import date

import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from dash_html_components.Datalist import Datalist


In [2]:
sns.set(rc={'figure.figsize':(10, 6)})

In [3]:
# read data from file
data = pd.read_excel("data//data.xlsx")
data.head()

,Document Date,Year/Month,Year,Month,supplier delivery date,delivery date,Company Code,Country,Purchasing Doc.,Item,...,open quantity,Delivery deviation in days,deviation indicator,deviation cause,deviation cause text,Net Value,Local Currency,Counter,Unnamed: 34,Unnamed: 35
0,2019-02-28,NaN,NaN,NaN,2019-03-07,2019-03-07,52,AT,8207406178,1,...,3,NaN,NaN,3,under-delivery,6720.0,EUR,1,NaN,NaN
1,2019-02-28,NaN,NaN,NaN,2019-03-07,2019-03-07,52,AT,8207406178,2,...,3,NaN,NaN,3,under-delivery,18162.0,EUR,1,NaN,NaN
2,2019-02-28,NaN,NaN,NaN,2019-03-07,2019-03-07,52,AT,8207406178,3,...,3,NaN,NaN,3,under-delivery,5710.0,EUR,1,NaN,NaN
3,2019-02-28,NaN,NaN,NaN,2019-03-07,2019-03-07,52,AT,8207406178,4,...,3,NaN,NaN,3,under-delivery,29890.0,EUR,1,NaN,NaN
4,2019-02-28,NaN,NaN,NaN,2019-03-07,2019-03-07,52,AT,8207406178,5,...,3,NaN,NaN,3,under-delivery,9646.0,EUR,1,NaN,NaN


# Datenanpassungen

In [4]:
#dropping some columns since unnecessaary 
data.drop("Unnamed: 34", inplace=True, axis = 1) 
data.drop("Unnamed: 35", inplace=True, axis = 1) 
data.drop("Delivery deviation  in days", inplace=True, axis = 1)

In [5]:
data["Delivery deviation in days"] = data["delivery date"] - data["supplier delivery date"]

In [6]:
data["Delivery deviation in days"] = data["Delivery deviation in days"].apply(lambda x: float(x.days))

In [7]:
conditions = [(data['Delivery deviation in days'] <= 0),
              (data['Delivery deviation in days'] >= 1) & (data['Delivery deviation in days'] <= 4),
              (data['Delivery deviation in days'] >= 5) & (data['Delivery deviation in days'] <= 10),
              (data['Delivery deviation in days'] > 10)]

values = ['in time', 'late: < 5 days', 'late: 5 to 10 days', 'late: > 10 days']


data['deviation indicator'] = np.select(conditions, values)

In [8]:
data['Year'] = data['Document Date'].dt.year
data['Month'] = data['Document Date'].dt.month
data['Year/Month'] = data['Document Date'].dt.year.astype(str)+'/'+data['Document Date'].dt.month.astype(str)

In [9]:
data.to_csv("data_01.csv")

# Seite 1: Lieferantenperformance (Visualisierung)

Gesamtsumme Ordered Spend & Anzahl Bestellungen (aktuel vs vorjahr)

In [10]:
#orderedSpend_year = data.groupby(["Year"])[["Net Value"]].sum().reset_index()

#number of orders not in time
orders_notIntime = data[data['deviation indicator'] != "in time"]
orders_notIntime_2020 = orders_notIntime[orders_notIntime["Year"]==2020]
numbOrders_notIntime_2020 = len(orders_notIntime_2020)
print(numbOrders_notIntime_2020)


29


In [11]:
#number of orders not in time in %
orders_2020 = data[data["Year"]==2020]
numbOrders_2020 = len(orders_2020)
numOrd_nIt_2020_percent = ((numbOrders_notIntime_2020 / numbOrders_2020) * 100)
print(numOrd_nIt_2020_percent, "%")

0.6909697402906838 %


Gesamtsumme Ordered Spend & Anzahl Bestellungen der abweichenden Lieferungen nach Abweichungsgründen (deviation cause) & Höhe der Verzögerung (deviation indicator)

In [12]:
deviationCause_deviationDays = orders_notIntime.groupby(["deviation cause"])[["Delivery deviation in days"]].sum().reset_index()
deviationCause_deviationDays

,deviation cause,Delivery deviation in days
0,0,75.0
1,5,28.0
2,7,656.0
3,8,7.0
4,9,2.0
5,10,18.0


Abweichende Lieferungen (Ordered Spend & Anzahl Bestellungen) im aktuellen Jahr (Document Year, YTD) nach Monaten (Document Month/Year)

In [13]:

numDeviation_month_2020 = orders_notIntime_2020.groupby(["Month"])['deviation indicator'].count().reset_index()
numDeviation_month_2020
#numbOrders_month_2019 = df2019_without_purchDoc_duplicates.groupby(["Month"])[["Purchasing Doc."]].count().reset_index().rename(columns={"Purchasing Doc.":"numbOrders"})

,Month,deviation indicator
0,1,13
1,2,10
2,4,2
3,5,4


Abweichende Lieferungen (Ordered Spend & Anzahl Bestellungen) im aktuellen Jahr (YTD) nach Einkaufsorganisation (Purchasing Org.)

In [14]:

numDeviation_Org_2020 = orders_notIntime_2020.groupby(["Purchasing Doc."])['deviation indicator'].count().reset_index()
numDeviation_Org_2020


,Purchasing Doc.,deviation indicator
0,8207889750,2
1,8207890736,2
2,8207900183,1
3,8207900184,1
4,8207914670,2
5,8207914671,2
6,8207928920,1
7,8207935452,2
8,8207935454,2
9,8207965091,2


Abweichende Lieferungen (Ordered Spend & Anzahl Bestellungen) der Top 10 Lieferanten nach Ordered Spend (aktuelles Jahr)

In [15]:
numDeviation_Org_2020.sort_values(by=['deviation indicator'])

,Purchasing Doc.,deviation indicator
16,8208056836,1
2,8207900183,1
3,8207900184,1
13,8207979083,1
12,8207968961,1
6,8207928920,1
14,8208029380,2
11,8207965124,2
10,8207965110,2
0,8207889750,2
